In [9]:
import os
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from tools import preproc

from tools.maneger import DataManager
import spacy
import pprint
import random

import copy
import numpy as np

from sklearn.model_selection import train_test_split


In [10]:
pre = preproc.Preprocessor()
nlp = spacy.load('ja_ginza')
def div2sentence(text):
    if isinstance(text, str):
        doc = nlp(text)
        sentence_list = [str(s)  for s in doc.sents]

    elif isinstance(text, list):
        sentence_list  = []
        docs = list(nlp.pipe(text, disable=['ner']))
            # return [ self.get_POS(sen_) for sen_ in sen]
        for doc in docs:
            sentence_list.extend( [str(s) for s in doc.sents] )
    else:
        return None
    
    return sentence_list 

300


In [3]:
def make_token_set(texts):
    token_set = set()
    docs = list(nlp.pipe(texts, disable=['ner']))
    for doc in docs:
        for token in doc:
            token_set.add(token.text)
    return token_set

In [4]:
def get_ave_length(texts):
    docs = list(nlp.pipe(texts, disable=['ner']))
    ave_length = 0
    for doc in docs:
        ave_length += len(doc)
    ave_length = int(ave_length/len(docs)) + 1
    # ave_length = ave_length/len(doc)
    return ave_length

In [5]:
def make_random_sentence(token_list, length):
    samples = random.choices(token_list, k=length)
    return "".join(samples)

In [6]:
root = "/".join(os.getcwd().split("/")[:-1])
print("root:", root)
path = root + '/error_category_classification/dbdc5_ja_dev_labeled/'
datalist = ['DCM', 'DIT', 'IRS']
error_types = ['Unclear intention', 'Wrong information',
 'Ignore question', 'Topic transition error', 
 'Lack of information', 'Repetition', 
 'Contradiction', 'Self-contradiction',
  'Lack of common sense', 'Semantic error',
   'Grammatical error', 'Ignore proposal', 
   'Ignore offer', 'Lack of sociality', 
   'Uninterpretable', 'Ignore greeting', 
   'No-Err']

root: /home/yamada/Documents/MMI/UNI


In [11]:
df = pre.read_json_with_NoErr(path, datalist)
print(df.shape)

(2000, 5)


In [12]:
vec_mode = "ginza"
data_mode_list = ["dialogue", "wiki"]
data_mode = data_mode_list[0]
data_path = "./X_y_data/{0}/".format(data_mode)
dataM = DataManager(data_path)
print(data_path)
way = ["LR", "DNN"]
data_name = "data_{0}.pickle".format(way[1])
print(data_name)

model_path = "./models/{0}/".format(data_mode)
modelM = DataManager(model_path)


./X_y_data/dialogue/
data_DNN.pickle


In [13]:
usr_list = []
for d, u, s, ec in zip(df.did, df.usr, df.sys, df.ec):
    usr_list.append(u)

In [30]:
text = copy.deepcopy(usr_list[:20])
text = div2sentence(text)
# pprint.pprint(text[:20])

In [35]:
# word2vec モデル作成
from gensim.models import word2vec
normalized_list = pre.noun2normal(text)
normalized_text = []
for nor in normalized_list:
    normalized_text.extend(nor)
normalized_list

[['そう', 'です', 'ね', '。'],
 ['名詞-普通名詞-副詞可能', 'とても', '暑い', 'です', 'から', '。'],
 ['名詞-普通名詞-副詞可能', 'に', '行き', 'たい', 'と', '思い', 'ます', '。'],
 ['はい', '。'],
 ['代名詞', 'は', '名詞-普通名詞-一般', 'に', '行き', 'ます', 'か', '？'],
 ['何故', 'です', 'か', '？'],
 ['そう', 'です', 'か', '。'],
 ['名詞-普通名詞-一般', 'に', '行く', 'の', 'は', 'どう', 'でしょう', 'か', '？'],
 ['名詞-普通名詞-一般', 'は', 'お', '好き', 'な', 'の', 'です', 'か', '？'],
 ['名詞-普通名詞-一般',
  'の',
  '名詞-普通名詞-サ変可能',
  'の',
  '名詞-普通名詞-一般',
  'は',
  '名詞-普通名詞-一般',
  'です',
  'か',
  '？'],
 ['代名詞', 'に', '行く', 'と', 'いい', 'です', 'か', '？'],
 ['名詞-普通名詞-副詞可能', 'は', 'とても', '暑く', 'なる', 'みたい', 'です', 'ね', '。'],
 ['涼しく',
  'なっ',
  'て',
  'き',
  'たら',
  '、',
  '名詞-普通名詞-サ変可能',
  'に',
  '名詞-普通名詞-一般',
  'へ',
  '行き',
  'たい',
  'です',
  'ね',
  '。'],
 ['美味しい', 'です', 'ね', '。'],
 ['で', 'も', '高い', 'です', 'ね'],
 ['名詞-数詞', '名詞-普通名詞-助数詞可能', 'ぐらい', 'か', 'な', '？'],
 ['名詞-普通名詞-サ変可能', 'が', '分かり', 'ませ', 'ん'],
 ['名詞-普通名詞-一般', 'が', '好き', 'です'],
 ['名詞-普通名詞-一般', 'の', '名詞-普通名詞-一般', 'は', '？'],
 ['好き', 'な', '名詞-普通名詞-一般', '接尾辞-名詞的-一般',

In [38]:
wv_model = word2vec.Word2Vec(normalized_text, vector_size=128, min_count=1, window=5, epochs=3)

In [45]:
word = "あ"
if word in wv_model.wv:
    wv_model.wv[word]
else:
    print("no")
    print(wv_model.predict_output_word(word))

no
None
